In [1]:
import pandas as pd
import numpy as np

In [4]:
diabetes = pd.read_csv('C:\IDE\Skillfactory\PY-14-Data_Cleaning\practic\\diabetes.csv')
diabetes.head()

# Признаки в данных:
# Pregnancies — количество беременностей.
# Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.
#BloodPressure — диастолическое артериальное давление (мм рт. ст.).
#SkinThickness — толщина кожной складки трицепса (мм).
# Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).
# BMI — индекс массы тела (вес/рост)
# DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).
# Age — возраст.
# Outcome — наличие диабета (0 — нет, 1 — да).

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [5]:
# Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте 
# все признаки в данных. Сколько записей осталось в данных?

diabetes = diabetes.drop_duplicates()
diabetes.shape[0]

768

In [37]:
# Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности 
# возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ 
# запишите имена признаков, которые вы нашли (без кавычек).

low_information_col = []

for col in diabetes.columns:
    top_freq = diabetes[col].value_counts(normalize=True).max()
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    if top_freq > 0.95:
        low_information_col.append(col)
    if nunique_ratio > 0.95:
        low_information_col.append(col)
        
print(low_information_col)
diabetes = diabetes.drop(low_information_col, axis=1)

[]


In [49]:
# Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. 
# Его вы можете взять из библиотеки numpy: np.nan.
#Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in diabetes.columns:
    if col in cols:
        diabetes[col] = diabetes[col].apply(lambda x: np.nan if x == 0 else x)


round(diabetes['Insulin'].isnull().mean(), 2)
# ИЛИ
diabetes.isnull().mean().round(2).sort_values(ascending=False)

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [57]:
# Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных 
# (с учетом удаленных неинформативных признаков в задании 8.2)?

thresh = diabetes.shape[0]*0.7
diabetes = diabetes.dropna(thresh=thresh, axis=1)
print(diabetes.shape[1])

8


In [63]:
# Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее 
# число записей в таблице?

m = diabetes.shape[1]
diabetes = diabetes.dropna(thresh=m-2, axis=0)
diabetes.shape[0]

761

In [69]:
# В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? 
# Ответ округлите до десятых.

values = {
    'Insulin': diabetes['Insulin'].median(),
    'SkinThickness': diabetes['SkinThickness'].median(),
    'BloodPressure': diabetes['BloodPressure'].median(),
    'Glucose': diabetes['Glucose'].median(),
    'BMI': diabetes['BMI'].median()
}

diabetes = diabetes.fillna(values)

diabetes['SkinThickness'].mean()

KeyError: 'Insulin'

In [68]:
null_data = diabetes.isnull().sum()
cols = null_data[null_data>0].index
for col in cols:
    diabetes[col] = diabetes[col].fillna(diabetes[col].median())
print(diabetes['SkinThickness'].mean().round(1))

29.1


In [71]:
# Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [73]:
# Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 4
Результирующее число записей: 757


In [75]:
# На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень 
# похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в 
# признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.

# Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять 
# не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

outliers, cleaned = outliers_iqr(diabetes, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(data = diabetes, feature = 'DiabetesPedigreeFunction', left = 3, right = 3, log_scale=True)
print(outliers.shape[0])

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732
0
